In [1]:
#! apt-get update -y

In [2]:
#! apt-get install wget -y

In [3]:
#! apt-get install unzip -y

In [4]:
#! wget LoanStats_web.csv.zips://storage.googleapis.com/19sep/LoanStats_web.csv.zip

In [5]:
#! unzip LoanStats_web.csv.zip

In [6]:
from pyspark.sql import functions as sparkf

In [7]:
from pyspark.sql import SparkSession

# สร้าง SparkSession ใหม่
spark = (
    SparkSession.builder
        # กำหนดชื่อแอปพลิเคชัน
        .appName("Integration of onlinePassenger and Station")
        # กำหนด URL ของ Spark master
        .master("spark://spark-master:7077")
        # กำหนดจำนวน memory ที่ executor จะใช้
        .config("spark.executor.memory", "1000m")
        # กำหนดจำนวน cores ที่ executor จะใช้
        .config("spark.executor.cores", "2")
        # กำหนดจำนวน cores สูงสุดที่ Spark จะใช้
        .config("spark.cores.max", "6")
        # สร้าง SparkSession ถ้ายังไม่มี, ถ้ามีแล้วจะใช้ SparkSession ที่มีอยู่
        .config("spark.jars.packages", "com.microsoft.azure:spark-mssql-connector:1.0.2") \
        .config("spark.sql.debug.maxToStringFields", 200) \

        .getOrCreate()
)

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.microsoft.azure#spark-mssql-connector added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ec9ffb05-a971-453e-a315-c58470940f13;1.0
	confs: [default]
	found com.microsoft.azure#spark-mssql-connector;1.0.2 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 100ms :: artifacts dl 2ms
	:: modules in use:
	com.microsoft.azure#spark-mssql-connector;1.0.2 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwn

In [8]:
# ใช้ SparkSession ที่เราสร้างขึ้น (spark) ในการอ่านข้อมูล

loanStat_raw_df = (
    spark.read  # ใช้วิธีการอ่านข้อมูล (read method)
        .format("csv")  # กำหนดรูปแบบของไฟล์ที่จะอ่านเป็น CSV
        .option("header", "true")  # กำหนดว่าไฟล์ CSV มี header ที่บรรทัดแรก
        .load("LoanStats_web.csv")  # โหลดข้อมูลจากไฟล์
)

In [10]:
loanStat_raw_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: string (nullable = true)
 |-- funded_amnt: string (nullable = true)
 |-- funded_amnt_inv: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string 

In [11]:
from pyspark.sql import DataFrame

def validate_dataframe_shape(df: DataFrame, expected_num_columns: int, expected_num_rows: int):
    # ตรวจสอบจำนวน column
    actual_num_columns = len(df.columns)
    if actual_num_columns != expected_num_columns:
        print(f"Unexpected number of columns. Expected: {expected_num_columns}, but got: {actual_num_columns}")
    else:
        print(f"Number of columns is as expected: {actual_num_columns}")

    # ตรวจสอบจำนวน row
    actual_num_rows = df.count()
    if actual_num_rows != expected_num_rows:
        print(f"Unexpected number of rows. Expected: {expected_num_rows}, but got: {actual_num_rows}")
    else:
        print(f"Number of rows is as expected: {actual_num_rows}")

# ตัวอย่างการใช้
expected_columns = 150
expected_rows = 1000
validate_dataframe_shape(loanStat_raw_df, expected_columns, expected_rows)


Unexpected number of columns. Expected: 150, but got: 144


Unexpected number of rows. Expected: 1000, but got: 1432466
